This notebook follows from the tutorials of CHGNet to relax LLZO-Li slabs that have been made in `llzo_li_balanced_sliced` directory.


LLZO‖Li Interface Relaxation Notebook
========================================

Each notebook handles only one structure.
1. Purpose: (This keeps updating)
- Relax a single LLZO‖Li (in this notebook LLZO_001_Zr_code93_sto__Li_110_slab_heavy) heterostructure using CHGNet
- Perform multi-stage optimization (in this notebook, CG → FIRE)
- Freeze bulk-like regions (15 Å at both ends)
- after all this, Relax lattice vectors to relieve interface strain

2. This notebook handles:
- Structure: LLZO_110_Li_order17_off__Li_100_slab_heavy
- Initial lattice height: 86.02 Å
- Number of atoms: 738

3. Method:
- CHGNet (v0.4.0) + ASE interface
- Stage 1: SciPyFminCG (no cell relaxation) → fmax target ~0.15 eV/Å
- Stage 2: FIRE (with optional cell relaxation) → fmax target ~0.05 eV/Å
- FrechetCellFilter used for combined force + stress minimization

4. Constraints:
- LLZO base: frozen bottom 14.25 Å
- Li top: frozen top 14.25 Å
- Only interfacial region relaxed
- Cell relaxation via `relax_cell=True` and `relax_cell_atoms="unconstrained"`

5. Outputs: (This will be decided later)
- relaxed_[structure_name].cif
- relaxed_[structure_name].traj
- (Optional) relaxation_log.pkl with energies, forces

6. Visual checks:
- Compare pre- and post-relaxation structures
- Ensure no Li diffusion into LLZO (via z-analysis)
- Confirm convergence (fmax < 0.05 eV/Å)

Author: Mehul Darak

Date: 15-07-2025


In [2]:
from pymatgen.core import Structure
import os

# Load structure
structure_path = "/home/mehuldarak/summer/llzo_li_balanced_sliced/LLZO_110_Li_order17_off__Li_100_slab_heavy.cif"  # replace with your file
structure = Structure.from_file(structure_path)

# Extract info
structure_name = os.path.basename(structure_path).replace(".cif", "")
lattice_height = structure.lattice.c
num_atoms = len(structure)

# Print output
print(f"- Structure: {structure_name}")
print(f"- Initial lattice height: {lattice_height:.2f} Å")
print(f"- Number of atoms: {num_atoms}")


- Structure: LLZO_110_Li_order17_off__Li_100_slab_heavy
- Initial lattice height: 86.02 Å
- Number of atoms: 738


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 1 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [3]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file("/home/mehuldarak/summer/llzo_li_balanced_sliced/LLZO_110_Li_order17_off__Li_100_slab_heavy.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 28.82 Å
Lowest Li slab atom: 42.20 Å
LLZO top z: 41.68 Å
Li penetration into LLZO: -0.52 Å


In [4]:
from pymatgen.core import Structure
import numpy as np

# Load structure
structure = Structure.from_file("/home/mehuldarak/summer/llzo_li_balanced_sliced/LLZO_110_Li_order17_off__Li_100_slab_heavy.cif")

# Get z-coordinates and element types
z_coords = np.array([site.z for site in structure.sites])
species = np.array([site.species_string for site in structure.sites])

# LLZO: non-Li atoms (La, Zr, O)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()
llzo_bottom = llzo_z.min()
llzo_thickness = llzo_top - llzo_bottom

# Li slab: Li atoms ABOVE LLZO (i.e. in metallic Li layer)
li_slab_z = np.array([
    site.z for site in structure.sites
    if site.species_string == "Li" and site.z > llzo_top
])
li_thickness = li_slab_z.ptp() if len(li_slab_z) > 0 else 0
li_bottom = li_slab_z.min() if len(li_slab_z) > 0 else None

# Penetration check
penetration = llzo_top - li_bottom if li_bottom is not None else 0

# Report
print(f"LLZO slab thickness: {llzo_thickness:.2f} Å")
print(f"Li slab thickness:   {li_thickness:.2f} Å")
print(f"LLZO top z:          {llzo_top:.2f} Å")
print(f"Lowest Li atom z:    {li_bottom:.2f} Å" if li_bottom else "No Li slab atoms found")
print(f"Li penetration into LLZO: {penetration:.2f} Å")


LLZO slab thickness: 26.15 Å
Li slab thickness:   28.82 Å
LLZO top z:          41.68 Å
Lowest Li atom z:    42.20 Å
Li penetration into LLZO: -0.52 Å


In [5]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.constraints import FixAtoms
from chgnet.model.dynamics import CHGNetCalculator, StructOptimizer
from ase.io import read, write
import numpy as np

# --- Load structure ---
structure = read("/home/mehuldarak/summer/llzo_li_balanced_sliced/LLZO_110_Li_order17_off__Li_100_slab_heavy.cif")

# --- Get z coordinates ---
z_coords = structure.get_positions()[:, 2]
z_min, z_max = z_coords.min(), z_coords.max()

# --- Define freeze zones ---
freeze_thickness_llzo = 22  # in Å
freeze_thickness_li = 23.5  # in Å
llzo_z_threshold = z_min + freeze_thickness_llzo
li_z_threshold = z_max - freeze_thickness_li

# --- Freeze LLZO base and Li top ---
freeze_mask = (z_coords < llzo_z_threshold) | (z_coords > li_z_threshold)
structure.set_constraint(FixAtoms(mask=freeze_mask))
print(f"Freezing {np.sum(freeze_mask)} atoms out of {len(structure)}")

# --- Attach CHGNet calculator ---
calc = CHGNetCalculator(use_device="cuda")
structure.set_calculator(calc)

Freezing 617 atoms out of 738
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda


/tmp/ipykernel_78018/1947634833.py:27: FutureWarning: Please use atoms.calc = calc
  structure.set_calculator(calc)


In [ ]:
# Stage 1: CG
opt1 = StructOptimizer(model=calc, optimizer_class="LBFGS", use_device="cuda")
result1 = opt1.relax(structure, fmax=0.15, steps=300, relax_cell=False, verbose=True)

       Step     Time          Energy          fmax
LBFGS:    0 13:46:06    -2102.366323       96.252349
LBFGS:    1 13:46:07    -2116.626955       19.428857
LBFGS:    2 13:46:07    -2108.005790       71.445104
LBFGS:    3 13:46:08    -2115.141561       62.364928
LBFGS:    4 13:46:09    -2109.486082       64.854754
LBFGS:    5 13:46:09    -2115.794346      183.815847
LBFGS:    6 13:46:10    -2113.561327       77.601713
LBFGS:    7 13:46:11    -2123.959969       17.941534
LBFGS:    8 13:46:11    -2117.833640       55.053694
LBFGS:    9 13:46:12    -2118.908536       20.350999
LBFGS:   10 13:46:13    -2128.121255       14.466719
LBFGS:   11 13:46:13    -2127.382253       20.116459
LBFGS:   12 13:46:14    -2119.340501       44.849492
LBFGS:   13 13:46:14    -2123.749529       48.148810
LBFGS:   14 13:46:15    -2121.197156       52.211552
LBFGS:   15 13:46:16    -2122.010762       55.738895
LBFGS:   16 13:46:16    -2128.210111       44.132544
LBFGS:   17 13:46:17    -2120.241732       48.74

KeyboardInterrupt: 

: 

In [16]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result1["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=False, verbose=True)

/tmp/ipykernel_56255/3154571052.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 20:48:52    -2692.110083        0.136644
FIRE:    1 20:48:53    -2692.114134        0.134898
FIRE:    2 20:48:53    -2692.121899        0.131163
FIRE:    3 20:48:54    -2692.131689        0.127374
FIRE:    4 20:48:54    -2692.143505        0.118269
FIRE:    5 20:48:54    -2692.156503        0.111001
FIRE:    6 20:48:55    -2692.169332        0.103045
FIRE:    7 20:48:55    -2692.183511        0.093833
FIRE:    8 20:48:56    -2692.198703        0.089371
FIRE:    9 20:48:56    -2692.215414        0.088418
FIRE:   10 20:48:56    -2692.234826        0.092469
FIRE:   11 20:48:57    -2692.256939        0.100811
FIRE:   12 20:48:57    -2692.283272        0.097430
FIRE:   13 20:48:58    -2692.312812        0.083648
FIRE:   14 20:48:58    -2692.347078        0.082827
FIRE:   15 20:48:59    -2692.386071        0.076299
FIRE:   16 20:48:59    -2692.431647        0.071338
FIRE:   17 20:48:59    -2692.484819        0.076280
FIRE:   18 20:

In [17]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import write

# Extract final structure from result3 (FIRE)
final_structure_pmg = result2["final_structure"]  # assuming result2 = FIRE
final_structure_ase = AseAtomsAdaptor.get_atoms(final_structure_pmg)

# Save as CIF and ASE trajectory
write(f"{structure_name}.cif", final_structure_ase)
write(f"{structure_name}.traj", final_structure_ase)

print("✅ Final structure saved successfully.")

✅ Final structure saved successfully.


In [ ]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file("/home/mehuldarak/summer/relax/LLZO_021_Li_order3_off__Li_100_slab_heavy/LLZO_021_Li_order3_off__Li_100_slab_heavy.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 14.28 Å
Lowest Li slab atom: 25.28 Å
LLZO top z: 25.17 Å
Li penetration into LLZO: -0.12 Å


In [13]:
!nvidia-smi

/bin/bash: /home/mehuldarak/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Mon Jul 14 20:45:54 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:65:00.0 Off |                  Off |
| 30%   48C    P8              18W / 300W |  48654MiB / 49140MiB |      0%      Default |
|                                         | 

In [14]:
!sudo kill 52973

/bin/bash: /home/mehuldarak/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
[sudo] password for mehuldarak: 
sudo: a password is required
^C
